In [5]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [17]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

In [18]:
if not api_key:
    print('no api_key_found')
else: 
    print('API key found')

API key found


In [19]:
#creting instance of openai class
openai=OpenAI()



In [20]:
########################## EXTRACTING DATA FROM THE WEBSITE################################

headers={
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website : 
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title.string if soup.title else 'No title found'
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text=soup.body.get_text(separator="\n",strip=True)

In [21]:
# testing ## # ## # #
ed=Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [22]:
#  prompts for sending message to llm
system_prompt="you are an assistant that analyzes the contents of a website \
    and provides a short summary,ignoring text that might be navigation related.\
    respond in markdown."
# Now to defin the user prompt we have to take in the title and body of the website

def user_prompt_for(website):
    user_prompt=f"you are looking at a website titled {website.title}"
    user_prompt+="\n The contents of this webiste is as follows; \
        please provide a short summary of this website in markdown.\
        if it includes news or announcements, then summarize these too.\n\n" # use of '\' = "Even though I'm breaking this line in the code editor, it's actually one single long line of text."
    user_prompt+=website.text
    return user_prompt

In [23]:
print(user_prompt_for(ed))

you are looking at a website titled Home - Edward Donner
 The contents of this webiste is as follows;         please provide a short summary of this website in markdown.        if it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI start

In [24]:
# # THE API FROM OPEN AI EXPECT MESSAGE IN A PARTICULAR WAY
# [
#     {"role":"system","content":"system message here"},
#     {"role":"user","content":"user prompt here"}
# ]

def message_for(website):
    return[
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt_for(website)}
    ]


In [25]:
message_for(ed)

[{'role': 'system',
  'content': 'you are an assistant that analyzes the contents of a website     and provides a short summary,ignoring text that might be navigation related.    respond in markdown.'},
 {'role': 'user',
  'content': 'you are looking at a website titled Home - Edward Donner\n The contents of this webiste is as follows;         please provide a short summary of this website in markdown.        if it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a fie

In [26]:
### BINDING IT ALL TOGETHER #################
def summarizer(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_for(website)
    )
    return response.choices[0].message.content



In [27]:
summarizer("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website features his interests in coding and experimenting with large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on using AI to help individuals discover their potential and engage with talent in the recruitment field. His background includes founding an AI startup, untapt, which was acquired in 2021.\n\n### Recent News and Announcements\n- **May 28, 2025:** Announcement of courses aimed at becoming an LLM expert and leader.\n- **May 18, 2025:** Information about the 2025 AI Executive Briefing.\n- **April 21, 2025:** Launch of the "Complete Agentic AI Engineering Course."\n- **January 23, 2025:** An upcoming LLM Workshop focusing on hands-on experience with agents and associated resources.\n\nOverall, the website serves as a platform for sharing insights, courses, and personal updates related to LLMs and AI in the recruitment sector.'

In [28]:
# make it nicer to read

def display_summary(url):
    summary=summarizer(url)
    display(Markdown(summary))

In [29]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website highlights his passion for coding, experimenting with large language models (LLMs), and his background as the co-founder and CTO of **Nebula.io**, an AI-driven platform aimed at enhancing talent discovery and management. He has previously founded **untapt**, an AI startup acquired in 2021. 

### Notable Content
- **About Ed:** Ed shares his interests in DJing, electronic music production, and engaging with tech discussions on platforms like Hacker News.
- **Professional Background:** He has experience in using proprietary LLMs tailored for talent management and has achieved recognition for his work through customer satisfaction and media coverage.

### News and Announcements
- **May 28, 2025:** Announcement for a course titled "Connecting my courses – become an LLM expert and leader."
- **May 18, 2025:** Information about the "2025 AI Executive Briefing."
- **April 21, 2025:** Details on "The Complete Agentic AI Engineering Course."
- **January 23, 2025:** Announcement regarding the "LLM Workshop – Hands-on with Agents – resources."

For those interested in connecting, Ed provides contact information and encourages engagement with his content.